# Clasificación de sentimientos con aprendizaje de pocos ejemplos (Few-shot Learning)

En este notebook probaé prompting con 0-shot, 1-shot y 5-shot usando openAI

In [2]:
import os
import openai
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, f1_score
from dotenv import load_dotenv
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

Usaré un conjunto simple con frases cortas etiquetas como "positivo" o "negativo"

In [5]:
examples = [
    ("I love this movie, it's fantastic!", "positivo"),
    ("This is the worst thing I've ever seen.", "negativo"),
    ("Absolutely amazing performance!", "positivo"),
    ("Terrible. I regret watching this.", "negativo"),
    ("I'm very happy with the result.", "positivo"),
    ("It's not good at all.", "negativo"),
    ("I enjoyed every minute of it.", "positivo"),
    ("I hate how this ended.", "negativo"),
]

# Dividimos en 5 ejemplos para few-shot y el resto como test
few_shot_examples = examples[:5]
test_data = examples[5:]

Función para construir prompts

In [6]:
def build_prompt(task_instruction, examples, new_input):
    """
    Construye un prompt con la instrucción, ejemplos (shots) y nueva entrada.
    """
    prompt = task_instruction + "\n\n"
    for ex, label in examples:
        prompt += f"Entrada: {ex}\nSentimiento: {label}\n\n"
    prompt += f"Entrada: {new_input}\nSentimiento:"
    return prompt

Función para llamar a OpenAI

In [ ]:
def classify_sentiment(prompt):
    """
    Llamamos a la API de OpenAI para completar el prompt y devolver la respuesta
    """
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": prompt}
        ],
        temperature=0
    )
    output = response['choices'][0]['message']['content']
    return output.strip().lower()


Me falta completar más código: probar las configuraciones 0-shot, 1-shot, 5-shot y la visualización de resultados